In [ ]:
#10.1 Summarization

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.chains.summarize import load_summarize_chain
import os

api_key = ''
os.environ['OPENAI_API_KEY'] = api_key

llm = ChatOpenAI(openai_api_key=api_key)

loader = WebBaseLoader("https://medium.com/p/144223fe60c8")
chain = load_summarize_chain(llm, chain_type="stuff")
docs = loader.load()
chain.run(docs).split('.')

['This article discusses the basics of recommendation systems, which are used in various platforms such as social media, e-commerce, and entertainment apps',
 ' It explains the terminology, types of recommendations, algorithms used, common issues, and the importance of context in making recommendations',
 ' The article also introduces different types of recommendations, such as non-personalized, semi-personalized, personalized, and opinion-based recommendations',
 ' It provides an overview of algorithms used in recommendation systems, such as content-based recommendations and collaborative filtering',
 ' The article concludes by mentioning the major challenges faced by recommendation systems, including the cold start problem and the gray sheep problem',
 '']

In [ ]:
#10.2 Text Tagging and Classification

from langchain.chat_models import ChatOpenAI
from langchain.chains import create_tagging_chain

# Schema
schema = {
    "properties": {
        "complexity": {
            "type": "integer",
            "enum": [1, 2, 3, 4, 5],
            "description": "describes how complex is the text. A higher score means more complex statement",
        },
        "tense": {
            "type": "string",
            "enum": ["past","present","future"],
        },
        "mood":{
            "type":"string",
            "enum":["suprised",'awful','regretful','happy','angry','normal']
        },
        "sentiment":
        {
            "type":"string",
            "description":"describes the sentiment of sentence",
            "enum":["positive","negative"]

        },

    },
    "required": ["language", "tense","mood","sentiment","complexity"],
}

# LLM
llm = ChatOpenAI(openai_api_key=api_key)
chain = create_tagging_chain(schema, llm)

inp = "Hey ! I assume I saw you yesterday. You look gullible"
print(inp,'\n',chain.run(inp))

inp = "He was so meticulous but now is on a decline"
print(inp,'\n',chain.run(inp))

Hey ! I assume I saw you yesterday. You look gullible 
 {'complexity': '2', 'tense': 'past', 'mood': 'normal', 'sentiment': 'negative'}
He was so meticulous but now is on a decline 
 {'complexity': '3', 'tense': 'present', 'mood': 'normal', 'sentiment': 'negative'}


In [ ]:
#10.3 Named Entity Recognition
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_extraction_chain

# Schema
schema = {
    "properties": {
        "name": {"type": "string"},
        "time":{"type":"string"},
        "day":{"type":"string"}
    }
}

# Run chain
llm = ChatOpenAI(openai_api_key=api_key)
chain = create_extraction_chain(schema, llm)

inp = """Rajesh went to KFC last Thursday. I saw him around 5.30 PM"""
print(chain.run(inp))

[{'name': 'Rajesh', 'time': '5.30 PM', 'day': 'last Thursday'}]


In [ ]:
#10.4 Text Embeddings
from langchain.embeddings import OpenAIEmbeddings
embeddings_model = OpenAIEmbeddings()
embeddings = embeddings_model.embed_documents(
    [
        "He is a wrestler",
        "Let's dance",
        "What was that",
        'Hey you',
        'I love her'
    ]
)
len(embeddings), len(embeddings[0])

(5, 1536)

In [ ]:
  #10.5.2 Multi-Classification
  from langchain.prompts.few_shot import FewShotPromptTemplate
  from langchain.prompts.prompt import PromptTemplate

  examples = [
    {"input":"football is a fun game",
     "output":"sports"},
    {"input":"SRK is the best actor",
     "output":"cinema"},
    {"input":"AI startups are at a boom right now",
     "output":"tech"},
    {"input":"He is good player of basketball",
     "output":"sports"},
    {"input":"Computers are becoming powerful now",
     "output":"tech"},
    {"input":"GodFather is a higly rated movie",
     "output":"cinema"}
  ]

  example_prompt = PromptTemplate(input_variables=["input", "output"], template="input:{input}\n output:{output}")


  prompt = FewShotPromptTemplate(
      examples=examples,
      example_prompt=example_prompt,
      suffix="Question: {input}",
      input_variables=["input"]
  )

  from langchain.chat_models import ChatOpenAI
  from langchain.chains import LLMChain

  chain = LLMChain(llm=ChatOpenAI(openai_api_key=api_key), prompt=prompt)

In [ ]:
chain.run("India lost the Cricket world cup final")

'output:sports'

In [ ]:
#10.5.3 Example Selection

import os
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.llms import HuggingFaceHub
from langchain.chains import LLMChain


examples = [
    {"input":"football is a fun game",
     "output":"sports"},
    {"input":"SRK is the best actor",
     "output":"cinema"},
    {"input":"AI startups are at a boom right now",
     "output":"tech"},
    {"input":"He is good player of basketball",
     "output":"sports"},
    {"input":"Computers are becoming powerful now",
     "output":"tech"},
    {"input":"GodFather is a higly rated movie",
     "output":"cinema"}
  ]

example_prompt = PromptTemplate(input_variables=["input", "output"], template="input:{input}\n output:{output}")
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=25,
)

prompt = FewShotPromptTemplate(example_selector=example_selector,example_prompt=example_prompt,suffix="Question: {input}",input_variables=["input"])
chain=LLMChain(llm=ChatOpenAI(openai_api_key=api_key),prompt=prompt)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [ ]:
print(prompt.format(input="India lost the cricket final"))

input:football is a fun game
 output:sports

input:SRK is the best actor
 output:cinema

Question: India lost the cricket final


In [ ]:
#10.6 POS Tagging, Segmentation and more

from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain

template = """You are a NLP machine. Act as a {problem} machine and give output for the input : {text}"""

system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain = LLMChain(
    llm=ChatOpenAI(openai_api_key=api_key),
    prompt=chat_prompt
)

In [ ]:
chain.run({"problem":'POS Tagging',"text":"He is not a good boy"})

'He/PRP is/VBZ not/RB a/DT good/JJ boy/NN'

In [ ]:
chain.run({"problem":'segmentation',"text":"He is not a good boy"})

'Segmented Output: \n1. He \n2. is \n3. not \n4. a \n5. good \n6. boy'

In [ ]:
chain.run({"problem":'Dependency parsing',"text":"He is not a good boy"})

'Here is the dependency parsing output for the input sentence "He is not a good boy":\n\n1. He - nsubj(is)\n2. is - ROOT\n3. not - neg(good)\n4. a - det(boy)\n5. good - amod(boy)\n6. boy - attr(is)\n\nThis output represents the syntactic structure and relationships between the words in the sentence.'